## Project setup

In [1]:
import os
import nltk
from google.colab import drive

In [2]:
!pip install transformers seqeval datasets
!apt install git-lfs

     |████████████████████████████████| 2.9 MB 3.9 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 270 kB 57.8 MB/s 
     |████████████████████████████████| 3.3 MB 22.7 MB/s 
     |████████████████████████████████| 895 kB 72.0 MB/s 
     |████████████████████████████████| 636 kB 68.0 MB/s 
     |████████████████████████████████| 56 kB 4.4 MB/s 
     |████████████████████████████████| 125 kB 63.1 MB/s 
     |████████████████████████████████| 1.3 MB 72.4 MB/s 
     |████████████████████████████████| 243 kB 73.8 MB/s 
     |████████████████████████████████| 160 kB 71.8 MB/s 
     |████████████████████████████████| 271 kB 75.7 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=0f515411768f2a2399c68dbda21a71bc8ac44795c042d37aa2c68bdb54e4805a
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
  Attempting uninstall:

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
mount_folder = os.path.join('/content', 'drive')
project_path = os.path.join(mount_folder, 'MyDrive', 'projects', 'product-absa')
drive.mount(mount_folder)

Mounted at /content/drive


In [5]:
%cd {project_path}

/content/drive/MyDrive/projects/product-absa


## Data loader

In [6]:
# constants
empty_tokens = 'NULL'
annotations_tricky = [['implicit_aspect'], ['corrupted'], ['aspect_multi_occurrence']]
tags = {"non_aspect": "O", "beginning_aspect": "B", "inside_aspect": "I"} 
#tags_map = {"O": "O", "B": "B-ORG", "I": "I-ORG"}
tags_decoder = {0: "O", 1: "B", 2: "I"} 
tags_encoder = {v:k for k,v in tags_decoder.items()}



In [42]:
from src.models.aspect_extraction_classifier.data_parser import data_reader_aspect_extraction
# load annotated aspect dataset
df_aspects = data_reader_aspect_extraction()
print(f'df lenght : {len(df_aspects)}')
df_aspects

Raw dataset size 16837
Raw dataset size 16837
aspect words are spread in the sentences
aspect words are spread in the sentences
compose aspect not found sentence
aspect words are spread in the sentences
compose aspect not found sentence
aspect words are spread in the sentences
aspect words are spread in the sentences
aspect words are spread in the sentences
aspect words are spread in the sentences
aspect words are spread in the sentences
aspect words are spread in the sentences
aspect words are spread in the sentences
aspect not found in sentence
aspect words are spread in the sentences
aspect words are spread in the sentences
aspect not found in sentence
aspect words are spread in the sentences
aspect words are spread in the sentences
aspect not found in sentence
aspect words are spread in the sentences
aspect not found in sentence
aspect not found in sentence
aspect words are spread in the sentences
aspect words are spread in the sentences
aspect words are spread in the sentences
asp

,review_text,aspect,annotations
0,I charge it at night and skip taking the cord ...,"[cord, battery life]","[O, O, O, O, O, O, O, O, O, B, O, O, O, O, O, ..."
1,I bought a HP Pavilion DV4-1222nr laptop and h...,[NULL],"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,The tech guy then said the service center does...,"[service center, ""sales"" team, tech guy]","[O, B, I, O, O, O, B, I, O, O, O, O, O, O, O, ..."
3,I investigated netbooks and saw the Toshiba NB...,[NULL],"[O, O, O, O, O, O, O, O, O]"
4,The other day I had a presentation to do for a...,[NULL],"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
...,...,...,...
16832,Spinache rolls have lots of garlic.,"[rolls, garlic]","[O, B, O, O, O, B, O]"
16833,"I use to always get the turkey club, until rec...","[turkey club, bread]","[O, O, O, O, O, O, B, I, O, O, O, O, O, O, O, ..."
16834,"The decor is worth a mention, with plush seati...","[decor, bar]","[O, B, O, O, O, O, O, O, O, O, O, O, O, O, B, ..."
16835,The Food The menu is better suited to the snac...,"[Food, snacking, bar]","[O, B, O, O, O, O, O, O, O, B, O, O, O, B, O, ..."


In [40]:
# filter annotated aspect dataset
input_column = 'text_tokens'
annotation_column = 'annotations'
label_column = 'tags'
aspect_flag_colum = 'type_aspect'
df_aspects = df_aspects[~df_aspects.annotations.apply(lambda x: x in annotations_tricky)]
#df_aspects[annotation_column] = df_aspects[annotation_column].apply(lambda annotations: [tags_map[annotation] for annotation in annotations])
df_aspects[input_column] = df_aspects.review_text.apply(lambda text_: nltk.tokenize.word_tokenize(text_))
df_aspects[label_column] = df_aspects[annotation_column].apply(lambda annotations: [tags_encoder[annotation] for annotation in annotations])
df_aspects = df_aspects.reset_index()
del df_aspects['index']
print(f'total samples {len(df_aspects)}')
df_aspects

total samples 16089


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,review_text,aspect,annotations,text_tokens,tags
0,I charge it at night and skip taking the cord ...,"[cord, battery life]","[O, O, O, O, O, O, O, O, O, B, O, O, O, O, O, ...","[I, charge, it, at, night, and, skip, taking, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
1,I bought a HP Pavilion DV4-1222nr laptop and h...,[NULL],"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[I, bought, a, HP, Pavilion, DV4-1222nr, lapto...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,The tech guy then said the service center does...,"[service center, ""sales"" team, tech guy]","[O, B, I, O, O, O, B, I, O, O, O, O, O, O, O, ...","[The, tech, guy, then, said, the, service, cen...","[0, 1, 2, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, ..."
3,I investigated netbooks and saw the Toshiba NB...,[NULL],"[O, O, O, O, O, O, O, O, O]","[I, investigated, netbooks, and, saw, the, Tos...","[0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,The other day I had a presentation to do for a...,[NULL],"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[The, other, day, I, had, a, presentation, to,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...
16084,"In Short A dark, narrow takeout area gives way...","[takeout, space]","[O, O, O, O, O, O, B, O, O, O, O, O, O, O, O, ...","[In, Short, A, dark, ,, narrow, takeout, area,...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
16085,Spinache rolls have lots of garlic.,"[rolls, garlic]","[O, B, O, O, O, B, O]","[Spinache, rolls, have, lots, of, garlic, .]","[0, 1, 0, 0, 0, 1, 0]"
16086,"I use to always get the turkey club, until rec...","[turkey club, bread]","[O, O, O, O, O, O, B, I, O, O, O, O, O, O, O, ...","[I, use, to, always, get, the, turkey, club, ,...","[0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, ..."
16087,"The decor is worth a mention, with plush seati...","[decor, bar]","[O, B, O, O, O, O, O, O, O, O, O, O, O, O, B, ...","[The, decor, is, worth, a, mention, ,, with, p...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."


In [44]:
annotations_occurrences = Counter([str(sample) for sample in df_aspects.annotations.to_list()])
print("corrupted", annotations_occurrences["['corrupted']"])  # 267 -61
print("aspect_multi_occurrence", annotations_occurrences["['aspect_multi_occurrence']"])  # 383 -399
print("implicit_aspect", annotations_occurrences["['implicit_aspect']"])  # 0 - 274

corrupted 76
aspect_multi_occurrence 398
implicit_aspect 274


## Paser text and labels function

In [9]:
task = "ner" 
model_checkpoint = "roberta-base" # "distilbert-base-uncased". # distilroberta-base #roberta-base # albert-base-v1
model_name = model_checkpoint.split("/")[-1]
batch_size = 16
label_list = [tags_decoder[i] for i in range(0,len(tags_decoder))]
label_list

['O', 'B', 'I']

In [10]:
import random
import transformers
import pandas as pd
from transformers import AutoTokenizer, RobertaTokenizerFast

In [11]:
def tokenize_and_align_labels(examples, input_column, label_column, label_all_tokens, max_length=None, padding=False, truncation=False, tokenizer=None):
    """
     Source: HF
     This method add a new key in the tokenized_inputs dict with the allinged 
     lables using the tokenier method word_ids(). You can choose to methods to 
     to alling the label, ignoring/considering the subwords with same word_id.
     For instance:
     >> tags.    = [0, 1, 0, 0, 1]
     >> word_ids =                 [None, 0, 1,    1,    1, 2, 3, 4, None]
     >> labels_all_tokens_true   = [-101, 0, 1,    1,    1, 0, 0, 1, -101]
     >> labels_all_tokens_false  = [-101, 0, 1, -101, -101, 0, 0, 1, -101]
    """
    tokenized_inputs = tokenizer(examples[input_column].to_list(), truncation=truncation, is_split_into_words=True, padding=padding, max_length=max_length)
    #  ,return_tensors="pt").to("cuda")
    labels = []
    for i, label in enumerate(examples[label_column].to_list()):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [12]:
# Check subword tokenization size without truncation
# https://huggingface.co/transformers/model_doc/roberta.html --> This tokenizer has been trained to treat spaces like parts of the tokens (a bit like sentencepiece) so a word will be encoded differently whether it is at the beginning of the sentence (without space) or not.. You can get around that behavior by passing add_prefix_space=True when instantiating this tokenizer or when you call it on some text, but since the model was not pretrained this way, it might yield a decrease in performance.
label_all_tokens = True

if 'roberta' in model_name:
  print(f'tokenizer imported from roberta family')
  #tokenizer = RobertaTokenizerFast.from_pretrained(model_checkpoint, add_prefix_space=True) # add_prefix_space=True
  tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)
  # Whether or not to add an initial space to the input. This allows to treat the leading word just as any other word. (RoBERTa tokenizer detect beginning of words by the preceding space).
else:
  print(f'tokenizer imported from {model_name}')
  tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenized_inputs = tokenize_and_align_labels(df_aspects, input_column, label_column, label_all_tokens, max_length=False, tokenizer=tokenizer) 

tokenizer imported from roberta family


Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [13]:
print(tokenized_inputs.keys())
print(df_aspects[label_column].iloc[0])
print(tokenized_inputs['input_ids'][0])
print(tokenized_inputs['labels'][0])

dict_keys(['input_ids', 'attention_mask', 'labels'])
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 2, 0]
[0, 38, 1427, 24, 23, 363, 8, 14514, 602, 5, 13051, 19, 162, 142, 9, 5, 205, 3822, 301, 479, 2]
[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 2, 0, -100]


In [14]:
print(len(tokenized_inputs['input_ids'][0]))
print(len(tokenized_inputs['labels'][0]))

21
21


In [15]:
from collections import Counter
subword_token_lenghts = [len(i) for i in tokenized_inputs['input_ids']]
df = pd.DataFrame({'len': subword_token_lenghts})
df.describe()

,len
count,16089.000000
mean,22.072285
std,11.703385
min,3.000000
25%,13.000000
50%,20.000000
75%,28.000000
max,92.000000


In [16]:
df_aspects[aspect_flag_colum] = df_aspects.aspect.apply(lambda x: 'non_aspect' if x==['NULL'] else 'aspect')
print(f'Total size: {len(df_aspects)}')
Counter(df_aspects.type_aspect.to_list())

Total size: 16089


Counter({'aspect': 10966, 'non_aspect': 5123})

In [17]:
subword_token_threshold = 32
valid_samples = [True if subword_token_lenght<=subword_token_threshold else False for subword_token_lenght in subword_token_lenghts]
df_aspects = df_aspects[valid_samples]
print(f'Total size: {len(df_aspects)}')
Counter(df_aspects.type_aspect.to_list())

Total size: 13389


Counter({'aspect': 8534, 'non_aspect': 4855})

## Build data structures and partitions

In [18]:
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import numpy as np
from datasets import load_dataset, load_metric
import torch

In [19]:
metric = load_metric("seqeval")


class AspDat(Dataset):
  def __init__(self, items):
    self.items = items
  
  def __len__(self):
    return len(self.items.input_ids)
  
  def __getitem__(self, idx):
    return {k: torch.tensor(v[idx]) for k, v in self.items.items()}


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }



Downloading:   0%|          | 0.00/2.48k [00:00<?, ?B/s]

In [20]:
z = df_aspects[aspect_flag_colum].to_list()
df_aspects_train, df_aspects_test = train_test_split(df_aspects, test_size=0.2, stratify=z, random_state=10)
print(f'Train size:{len(df_aspects_train)}')
print(f'Test size:{len(df_aspects_test)}')

Train size:10711
Test size:2678


In [21]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
model_dropout = 0.15 # # todo update using the right method https://stackoverflow.com/questions/64947064/transformers-pretrained-model-with-dropout-setting
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list), attention_probs_dropout_prob=model_dropout, hidden_dropout_prob=model_dropout)

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForTokenClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able

## Build training and test datasets

In [22]:
max_length = subword_token_threshold # 64 - 32
items_train = tokenize_and_align_labels( 
                                  examples=df_aspects_train, 
                                  input_column=input_column, 
                                  label_column=label_column, 
                                  label_all_tokens=True,
                                  max_length=max_length, 
                                  padding='max_length',
                                  truncation=True,
                                  tokenizer=tokenizer)

items_test = tokenize_and_align_labels( 
                                  examples=df_aspects_test, 
                                  input_column=input_column, 
                                  label_column=label_column, 
                                  label_all_tokens=True,
                                  max_length=max_length, 
                                  padding='max_length',
                                  truncation=True,
                                  tokenizer=tokenizer)

In [23]:
len(items_train['input_ids'])

10711

In [47]:
# tengo algo mal aca
train_ = AspDat(items_train)
test_ = AspDat(items_test)
print(f'train size:{len(train_)} --- test_size:{len(test_)} ')

train size:10711 --- test_size:2678 


In [25]:
train_[0]

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]),
 'input_ids': tensor([   0,   38,   56,  127, 3034, 6777, 1490,   31, 6366, 5827,    4,  175,
          479,    2,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1]),
 'labels': tensor([-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100])}

In [26]:
#from transformers import AdamW, get_linear_schedule_with_warmup
#lr = 5e-5
#warmup_steps= 400
#weight_decay=0.01
#num_training_steps = 505
#optimizer = AdamW(model.parameters(), lr = lr, weight_decay=weight_decay)
#scheduler = get_linear_schedule_with_warmup(optimizer=optimizer, num_warmup_steps=warmup_steps, num_training_steps= num_training_steps)

# Model building

In [27]:
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "steps",
    learning_rate= 7e-5, #2e-5
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    warmup_steps=400,             
    num_train_epochs=3,
    weight_decay=0.08, #0.01
    logging_steps=50,
    lr_scheduler_type = 'cosine', # 'cosine_with_restarts' --> https://huggingface.co/transformers/_modules/transformers/trainer_utils.html 
    seed = 42,
    #output_dir='./results',
    #logging_dir='./logs',
    load_best_model_at_end=True
)

In [28]:
config = model.config
print(f'hidden_dropout_prob: {config.hidden_dropout_prob}')
print(f'attention_probs_dropout_prob: {config.attention_probs_dropout_prob}')

hidden_dropout_prob: 0.15
attention_probs_dropout_prob: 0.15


In [29]:
trainer = Trainer(model, 
                  args, 
                  train_dataset=train_, 
                  eval_dataset=test_,
                  #data_collator=data_collator, 
                  #tokenizer=tokenizer, 
                  compute_metrics=compute_metrics)

In [30]:
trainer.train()

***** Running training *****
  Num examples = 10711
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 504


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
50,0.689600,0.272905,0.333333,0.000270,0.000539,0.886908
100,0.214000,0.165170,0.604015,0.624697,0.614182,0.932434
150,0.151000,0.148294,0.661333,0.813427,0.729537,0.940909
200,0.139300,0.147020,0.661928,0.866271,0.750438,0.941643
250,0.130400,0.129975,0.724427,0.792397,0.756889,0.946899
300,0.120200,0.130878,0.711268,0.844163,0.772038,0.948272
350,0.117600,0.124988,0.704380,0.862766,0.775570,0.950047
400,0.111000,0.131811,0.704988,0.849825,0.770660,0.948343
450,0.108000,0.120807,0.734183,0.854139,0.789631,0.952131
500,0.100900,0.118939,0.737871,0.865193,0.796476,0.953314


***** Running Evaluation *****
  Num examples = 2678
  Batch size = 64
Saving model checkpoint to roberta-base-finetuned-ner/checkpoint-50
Configuration saved in roberta-base-finetuned-ner/checkpoint-50/config.json
Model weights saved in roberta-base-finetuned-ner/checkpoint-50/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2678
  Batch size = 64
Saving model checkpoint to roberta-base-finetuned-ner/checkpoint-100
Configuration saved in roberta-base-finetuned-ner/checkpoint-100/config.json
Model weights saved in roberta-base-finetuned-ner/checkpoint-100/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2678
  Batch size = 64
Saving model checkpoint to roberta-base-finetuned-ner/checkpoint-150
Configuration saved in roberta-base-finetuned-ner/checkpoint-150/config.json
Model weights saved in roberta-base-finetuned-ner/checkpoint-150/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2678
  Batch size = 64
Saving model checkpoint to rober

TrainOutput(global_step=504, training_loss=0.1874760349709836, metrics={'train_runtime': 335.2289, 'train_samples_per_second': 95.854, 'train_steps_per_second': 1.503, 'total_flos': 524770287135552.0, 'train_loss': 0.1874760349709836, 'epoch': 3.0})

In [31]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 2678
  Batch size = 64


{'epoch': 3.0,
 'eval_accuracy': 0.953314393939394,
 'eval_f1': 0.7964755522462149,
 'eval_loss': 0.1189391240477562,
 'eval_precision': 0.7378707748907795,
 'eval_recall': 0.8651927743327043,
 'eval_runtime': 4.703,
 'eval_samples_per_second': 569.429,
 'eval_steps_per_second': 8.931}

In [32]:
predictions, labels, _ = trainer.predict(test_)
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

***** Running Prediction *****
  Num examples = 2678
  Batch size = 64


{'_': {'f1': 0.7964755522462149,
  'number': 3709,
  'precision': 0.7378707748907795,
  'recall': 0.8651927743327043},
 'overall_accuracy': 0.953314393939394,
 'overall_f1': 0.7964755522462149,
 'overall_precision': 0.7378707748907795,
 'overall_recall': 0.8651927743327043}

In [33]:
model.parameters

<bound method Module.parameters of RobertaForTokenClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.15, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.15, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768

# Error analysis

In [34]:

number_errors_counter = 0
test_error = []
for i in range(0,len(df_aspects_test)):
  if true_predictions[i]!=true_labels[i]: 
    test_error.append(False)
    test_sample = test_[i]
    tokens_subwords = tokenizer.convert_ids_to_tokens(test_sample['input_ids'])
    text_tokens = [tokens_subwords[idx][1:] for idx, label in enumerate(test_sample['labels']) if label!=-100] # [1:] to hide the special space token of Roberta ¨G
    #print(f'Error in {text_tokens}')
    token_size = len(true_predictions[i])
    sentence_analysis = []
    for j in range(0,token_size):
      true_label = true_labels[i][j]
      true_prediction = true_predictions[i][j]
      if (true_prediction == true_label):
        if true_label=='O':
          sentence_analysis.append(('TN', text_tokens[j]))
        else:
          sentence_analysis.append(('TP',text_tokens[j]))
      else:
        if true_label == 'O':
          sentence_analysis.append(('FN',text_tokens[j]))
        else:
          sentence_analysis.append(('FP',text_tokens[j]))
    print(f'---------Test sample : {i} ----------------')
    print(f'Analysis : {sentence_analysis}')
    print(f"Predicted aspects {[text_tokens[idx] for idx, tag in enumerate(true_predictions[i]) if tag!='O']}")
    print(f"True aspects {[text_tokens[idx] for idx, tag in enumerate(true_labels[i]) if tag!='O']}")
  else:
    test_error.append(True)


      

---------Test sample : 3 ----------------
Analysis : [('TN', 'Not'), ('TN', 'to'), ('TN', 'be'), ('TN', 'overlooked'), ('TN', ','), ('TN', 'the'), ('FN', 'service'), ('TN', 'is'), ('TN', 'excellent'), ('TN', '.')]
Predicted aspects ['service']
True aspects []
---------Test sample : 8 ----------------
Analysis : [('TN', 'The'), ('FP', 'whole'), ('TP', 'fish'), ('TN', 'was'), ('TN', 'excellent'), ('TN', '--'), ('TN', 'not'), ('TN', 'overc'), ('TN', 'oked'), ('TN', 'and'), ('FP', 'served'), ('TN', 'in'), ('TN', 'a'), ('FP', 'light'), ('FP', 'tomato'), ('TP', 'broth'), ('TN', '.')]
Predicted aspects ['fish', 'tomato', 'broth']
True aspects ['whole', 'fish', 'served', 'light', 'tomato', 'broth']
---------Test sample : 14 ----------------
Analysis : [('TN', 'We'), ('TN', "'"), ('TN', 'e'), ('TN', 'always'), ('TN', 'gotten'), ('TN', 'amazing'), ('FN', 'service'), ('TN', 'and'), ('TN', 'we'), ('TN', 'love'), ('TN', 'the'), ('FN', 'food'), ('TN', '.')]
Predicted aspects ['service', 'food']
True

In [38]:
# Add analytical flags
df_aspects_test['aspect_size'] = df_aspects_test.aspect.apply(lambda x: len(x) if x!=['NULL'] else 0)
df_aspects_test['sentence_size'] = df_aspects_test.text_tokens.str.len()
df_aspects_test['composed_aspect'] = df_aspects_test.aspect.apply(lambda x: True if any([True if len(x_i.split(' '))>1 else False for x_i in x]) else False)
df_aspects_test['test_error'] = test_error


df_aspects_test_incorrect = df_aspects_test[~df_aspects_test.test_error]
df_aspects_test_correct = df_aspects_test[df_aspects_test.test_error]

df_aspects_empty = df_aspects_test[df_aspects_test.aspect_size==0]
df_aspects_empty_correct = df_aspects_empty[df_aspects_empty.test_error]
df_aspects_empty_incorrect = df_aspects_empty[~df_aspects_empty.test_error]

df_aspects_composed = df_aspects_test[df_aspects_test.composed_aspect]
df_aspects_composed_correct = df_aspects_composed[df_aspects_composed.test_error]
df_aspects_composed_incorrect = df_aspects_composed[~df_aspects_composed.test_error]

print('Stats correct predictions')
print(df_aspects_test_correct.describe())


print('Stats incorrect predictions')
print(df_aspects_test_incorrect.describe())

print('Stats empty aspect correct')
print(df_aspects_empty_correct.describe())

print('Stats empty aspect incorrect')
print(df_aspects_empty_incorrect.describe())

print('Stats composed aspect correct')
print(df_aspects_composed_correct.describe())

print('Stats composed aspect incorrect')
print(df_aspects_composed_incorrect.describe())
"""
- Analysis according sentence size
The sentences classified with errors has a high amount of words per sentence.
13.38 for correct classification and 17.13 for incorrect. Then we can expect
a best formance of the model for short sentence. (Could be a good idea
to split very long sentences (for instance more than 20 tokens) in short ones)

- Analysis regarding aspect with composed words
Analysing predictions that include composed words we found more errors in phrases 
that included longest composed word, the averaged size of aspect words for correct 
predictions is 1.68 and the incorrect 2.14 (size of composed words).
The 47% [254/(254+284)]of the composed aspect show errors in the predictions.
The composed aspect represents 9,4% of all errors in test (254/2578)


- Performance when non aspect:
Analysing the predicted test sentences without aspect. The sentences lenght in the 
correct and incorrect predictions shows a similar pattern. All the errors 
found in this "non aspect" group are errors type 1, knonwn as false positives (FP).
Analizing particular cases we found many reasons ("True aspects []" in the above
print analysis) 300/(671+300) = 0.30 -> 30% false positive rate that represents 
a 11% ( 300/2678 ) of all the errors in test.

* Anotations with erros, reviews was labeled without aspect but have an aspect. The algoritm works well in thoses cases the error come from the annotation process 
   A) Analysis : [('TN', 'The'), ('FN', 'flavors'), ('TN', 'robust'), ('TN', 'and'), ('TN', 'subtle'), ('TN', '.')]. 
      Predicted aspects ['flavors']
      True aspects []
   B) Analysis : [('TN', 'I'), ('TN', "'"), ('TN', 'e'), ('TN', 'waited'), ('TN', 'over'), ('TN', 'one'), ('TN', 'hour'), ('TN', 'for'), ('FN', 'food'), ('TN', '.')]
      Predicted aspects ['food']
      True aspects []
   C) Analysis : [('TN', 'It'), ('TN', "'"), ('TN', ''), ('TN', 'fast'), ('TN', ','), ('TN', 'light'), ('TN', ','), ('TN', 'and'), ('TN', 'simple'), ('TN', 'to'), ('FN', 'use'), ('TN', '.')]
      Predicted aspects ['use']
      True aspects []
   D) Analysis : [('TN', 'Thick'), ('FN', 'fries'), ('TN', ','), ('TN', 'meat'), ('TN', ''), ('FN', 'chili'), ('TN', 'and'), ('TN', 'stuffed'), ('FN', 'baked'), ('FN', 'potatoes'), ('TN', 'round'), ('TN', 'out'), ('TN', 'a'), ('FN', 'menu'), ('TN', 'that'), ('TN', 'includes'), ('TN', 'a'), ('TN', 'cool'), ('TN', ','), ('TN', 'ultra'), ('TN', ''), ('TN', 'h'), ('TN', 'ck'), ('FN', 'chocolate'), ('TN', 'Frost'), ('TN', ''), ('TN', '.')]
      Predicted aspects ['fries', 'chili', 'baked', 'potatoes', 'menu', 'chocolate']
      True aspects []
   E) Analysis : [('TN', 'The'), ('FN', 'portions'), ('TN', 'are'), ('TN', 'very'), ('TN', 'large'), ('TN', 'and'), ('TN', 'the'), ('FN', 'service'), ('TN', 'is'), ('TN', 'fantastic'), ('TN', '!'), ('TN', '!')]
      Predicted aspects ['portions', 'service']
      True aspects [] 

* Annotations with high level of subjectivity. Some mentioned aspect are not related directly with the product. The algoritm works well in thoses cases the error come from the annotation process becuase the phrase include explicit aspects
  A) Analysis : [('TN', 'I'), ('TN', 'tried'), ('TN', 'several'), ('FN', 'monitors'), ('TN', 'and'), ('TN', 'several'), ('FN', 'HDMI'), ('FN', 'cables'), ('TN', 'and'), ('TN', 'this'), ('TN', 'was'), ('TN', 'the'), ('TN', 'case'), ('TN', 'each'), ('TN', 'time'), ('TN', '.')]
     Predicted aspects ['monitors', 'HDMI', 'cables']
     True aspects []
  B) Analysis : [('TN', 'And'), ('TN', 'in'), ('TN', '6'), ('TN', 'months'), ('TN', ','), ('TN', 'there'), ('TN', 'have'), ('TN', 'been'), ('TN', 'no'), ('TN', 'freezing'), ('TN', 'up'), ('TN', 'and'), ('TN', 'no'), ('TN', 'blue'), ('TN', ','), ('TN', 'purple'), ('TN', ','), ('TN', 'black'), ('TN', 'or'), ('TN', 'any'), ('TN', 'other'), ('TN', 'type'), ('TN', 'of'), ('FN', 'screen'), ('TN', '.')]
     Predicted aspects ['screen']
     True aspects []


* The algorihm has problem to differentiate the main entity and their aspect (when the phrase has an entity but not entity-aspect). Probably using a list of "products" those errors can be filtered. Depending the application cotext it could not be an error (In case extract opinion entities and aspect are ok for you)
  A) Analysis : [('TN', 'Fantastic'), ('FN', 'place'), ('TN', '.')]
     Predicted aspects ['place']
     True aspects []
  B) Analysis : [('TN', 'The'), ('FN', 'OS'), ('TN', 'is'), ('TN', 'easy'), ('TN', ','), ('TN', 'and'), ('TN', 'offers'), ('TN', 'all'), ('TN', 'kinds'), ('TN', 'of'), ('TN', 'surprises'), ('TN', '.')]
     Predicted aspects ['OS']
     True aspects []
  C) Analysis : [('TN', 'this'), ('FN', 'place'), ('TN', 'is'), ('TN', 'amazingly'), ('TN', 'bad'), ('TN', '.')]
     Predicted aspects ['place']
     True aspects []
  D) Analysis : [('TN', 'Not'), ('TN', 'to'), ('TN', 'be'), ('TN', 'overlooked'), ('TN', ','), ('TN', 'the'), ('FN', 'service'), ('TN', 'is'), ('TN', 'excellent'), ('TN', '.')]
     Predicted aspects ['service']
     True aspects []
  E) Analysis : [('TN', 'One'), ('TN', 'of'), ('TN', 'the'), ('TN', 'best'), ('FN', 'hot'), ('FN', 'dogs'), ('TN', 'I'), ('TN', 'have'), ('TN', 'ever'), ('TN', 'eaten'), ('TN', '.')]
     Predicted aspects ['hot', 'dogs']
     True aspects []

"""

Stats correct predictions
       aspect_size  sentence_size
count  1765.000000    1765.000000
mean      1.048159      13.375637
std       1.019344       6.396301
min       0.000000       1.000000
25%       0.000000       8.000000
50%       1.000000      13.000000
75%       2.000000      18.000000
max       5.000000      30.000000
Stats incorrect predictions
       aspect_size  sentence_size
count   913.000000     913.000000
mean      1.414020      17.138007
std       1.220934       6.423863
min       0.000000       2.000000
25%       0.000000      12.000000
50%       2.000000      17.000000
75%       2.000000      22.000000
max       5.000000      30.000000
Stats empty aspect correct
       aspect_size  sentence_size
count        669.0     669.000000
mean           0.0      11.173393
std            0.0       5.873656
min            0.0       1.000000
25%            0.0       6.000000
50%            0.0      10.000000
75%            0.0      15.000000
max            0.0      30.000000
S

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

'\n- Analysis according sentence size\nThe sentences classified with errors has a high amount of words per sentence.\n13.38 for correct classification and 17.13 for incorrect. Then we can expect\na best formance of the model for short sentence. (Could be a good idea\nto split very long sentences (for instance more than 20 tokens) in short ones)\n\n- Analysis regarding aspect with composed words\nAnalysing predictions that include composed words we found more errors in phrases \nthat included longest composed word, the averaged size of aspect words for correct \npredictions is 1.68 and the incorrect 2.14 (size of composed words).\nThe 47% [254/(254+284)]of the composed aspect show errors in the predictions.\nThe composed aspect represents 9,4% of all errors in test (254/2578)\n\n\n- Performance when non aspect:\nAnalysing the predicted test sentences without aspect. The sentences lenght in the \ncorrect and incorrect predictions shows a similar pattern. All the errors \nfound in this "n